In [ ]:
from Game import *
from Game.minimax import *

Version:  0.3.02


## Nim 9 -- makes it easier to see the learning

In [ ]:
def initial_state(): 
    """ returns  - The initial state of the game"""
    return 9

def valid_moves(state,player):
    """returns  - a list of the valid moves for the state and player"""

    if state==2:
        return [1,2]
    elif state==1:
        return [1]
    else:
        return [1,2,3]

def show_state(state,player):
    """prints or shows the current state"""
    print(f"There are >>{state} sticks<<< for player {player}.")

def update_state(state,player,move):
    """returns  - the new state after the move for the player"""

    new_state=state-move

    return new_state


def win_status(state,player):
    """    returns  - 'win'  if the state is a winning state for the player, 
               'lose' if the state is a losing state for the player,
               'stalemate' for a stalemate
               None otherwise
    """

    if state==0:
        return 'lose'

    if state==1:
        return 'win'


    return None
    


## Agents

In [ ]:
def random_move(state,player):    
    moves=valid_moves(state,player)
    return random.choice(moves)

random_agent=Agent(random_move)

def human_move(state,player):
    print("Player ", player)
    valid_move=False
    while not valid_move:
        move=int(input('What is your move? '))

        if move in valid_moves(state,player):
            valid_move=True
        else:
            print("Illegal move.")

    return move

human_agent=Agent(human_move)

In [ ]:
def minimax_move(state,player):
    values,moves=minimax_values(state,player,display=False)
    return top_choice(moves,values)


minimax_agent=Agent(minimax_move)

In [ ]:
def print_table(S,start=""):
    S=skittles_agent.S
    for state in S:
        print(start,state,": ",end="")
        for action in [1,2,3]:
            if action in S[state]:
                print(S[state][action],end=" ")
            else:
                print("--",end=" ")

        print()

In [ ]:
def skittles_move(state,player,info):
    from IPython.display import display
    S=info.S
    last_state=info.last_state
    last_action=info.last_action
    verbose=info.verbose

    
    if verbose:
        print("\t","Player ",player," is thinking...")
        print("\t","State: ",state)
        print("\t","Table:")
        print_table(S,"\t")        
        print("\t","Last state,action: ",last_state,last_action)
    
    # make/adjust the table

    if state not in S:
        # add a row to the table for each move
        S[state]=Table()
        moves=valid_moves(state,player)
        for action in moves:
            S[state][action]=3  # number of skittles/beads for each move
    
        if verbose:
            print("\t","State ",state,"unknown...added to table")
            print("\t","Table")
            print_table(S,"\t")        
    
    move=weighted_choice(S[state])
    if verbose:
        print("\t","Choosing from S[",state,"]",S[state],"....Move: ",move)

    if move is None:  # there are no skittles in this row
        if last_state:
            if verbose:
                print("\t","No possible moves!")
                print("\t","Modifying the table: removing one skittle from (state,action) ",last_state,last_action)
            S[last_state][last_action]=S[last_state][last_action]-1
            
            if verbose:
                print("\t","Table:")
                print_table(S,"\t")        

            if S[last_state][last_action]<0:                
                S[last_state][last_action]=0
                if verbose:
                    print("\t","Negative skittles...fixing.")
                    print("\t","Table:") 
                    print_table(S,"\t")        

        else:
            if verbose:
                print("\t","Started in a bad state ",state,"with no moves and no last state.  Won't modify table.")
                  
                  

        move=random_move(state,player)

    
    return move

def skittles_after(status,player,info):
    S=info.S
    last_state=info.last_state
    last_action=info.last_action
    verbose=info.verbose

    if verbose:
        print("\t","End of Game adjustments")
        print("\t","Player ",player," is thinking...")
        print("\t","Win Status: ",status)
        print("\t","Table:",S)
        print("\t","Last state,action: ",last_state,last_action)
        
    if status=='lose':
        if last_state:
            S[last_state][last_action]=S[last_state][last_action]-1
            if S[last_state][last_action]<0:
                S[last_state][last_action]=0
                
            if verbose:
                print("\t","Modifying the table: removing one skittle from (state,action) ",last_state,last_action)
                print("\t","Table:",S)

                
        else:
            if verbose:
                print("\t","No last state, so nothing to learn.")
            
                
    else:
        if verbose:
            print("\t","No adjustments needed.")
        
    # does this double-count the learning if you lose on your own turn        
    

In [ ]:
skittles_agent=Agent(skittles_move)
skittles_agent.S=Table()
skittles_agent.post=skittles_after
skittles_agent.verbose=True

In [ ]:
def perfect_move(state,player):
    move=(state-1)%4

    if move==0:  # bad state
        move=1

    return move

perfect_agent=Agent(perfect_move)

In [ ]:
g=Game()
g.run(human_agent,skittles_agent)

====
Game  1
There are >>9 sticks<<< for player 1.
Player  1


What is your move?  1


Player 1 moves 1
There are >>8 sticks<<< for player 2.
	 Player  2  is thinking...
	 State:  8
	 Table:
	 Last state,action:  None None
	 State  8 unknown...added to table
	 Table
	 8 : 3 3 3 
	 Choosing from S[ 8 ] {1: 3, 2: 3, 3: 3} ....Move:  2
Player 2 moves 2
There are >>6 sticks<<< for player 1.
Player  1


What is your move?  3


Player 1 moves 3
There are >>3 sticks<<< for player 2.
	 Player  2  is thinking...
	 State:  3
	 Table:
	 8 : 3 3 3 
	 Last state,action:  8 2
	 State  3 unknown...added to table
	 Table
	 8 : 3 3 3 
	 3 : 3 3 3 
	 Choosing from S[ 3 ] {1: 3, 2: 3, 3: 3} ....Move:  3
Player 2 moves 3
There are >>0 sticks<<< for player 2.
Player  1 won.
	 End of Game adjustments
	 Player  2  is thinking...
	 Win Status:  lose
	 Table: {8: {1: 3, 2: 3, 3: 3}, 3: {1: 3, 2: 3, 3: 3}}
	 Last state,action:  3 3
	 Modifying the table: removing one skittle from (state,action)  3 3
	 Table: {8: {1: 3, 2: 3, 3: 3}, 3: {1: 3, 2: 3, 3: 2}}


[1]

In [ ]:
SaveTable(skittles_agent.S,'nim skittles.json')

In [ ]:
skittles_agent.S

{8: {1: 3, 2: 3, 3: 3}, 3: {1: 3, 2: 3, 3: 2}}